This notebook simply identfies which matches are most common so the interstate links can be matched, these segments being the hghest functional class are typically the closest match, but sometimes frontage roads get matched on accident. Therefore the next Chapter, which checks the interstate matches will be the most controlled check of matches.

In [1]:
import pandas as pd
import os

path_to_data = os.getcwd() + '\\Data\\'
matches = pd.read_csv(path_to_data + 'matches.csv')

modeling_attributes_df = pd.read_csv(path_to_data + 'modeling_attributes.csv')
modeling_type = modeling_attributes_df.filter(['modeling_id', 'modeling_type'])

osm_attributes_df = pd.read_csv(path_to_data + 'osm_attributes.csv')
osm_type = osm_attributes_df.filter(['osm_id', 'osm_type'])

modeling_type_to_matches = pd.merge(modeling_type, matches, on = 'modeling_id')
modeling_type_to_osm_type = pd.merge(osm_type, modeling_type_to_matches, on = 'osm_id')

modeling_type_to_osm_type.head()

,osm_id,osm_type,modeling_id,modeling_type,FID,distance,NEAR_RANK,NEAR_ANGLE,FROM_X,FROM_Y,NEAR_X,NEAR_Y
0,0,tertiary,27293,5,910013,15.675226,3,-50.222375,-105.252840,39.997379,-105.252981,39.997470
1,0,tertiary,27293,5,910014,0.340380,1,-93.388778,-105.253105,39.997886,-105.253109,39.997886
2,0,tertiary,27293,5,910017,0.819235,1,-90.300127,-105.253103,39.998463,-105.253113,39.998463
3,0,tertiary,27293,5,910021,2.671340,2,-89.825993,-105.253085,39.999041,-105.253116,39.999041
4,0,tertiary,27293,5,910023,4.270265,1,-90.501377,-105.253067,39.999620,-105.253117,39.999619


The cell above joined the modeling type, or functional class to each match between networks. The cell below will limit these matches to the closest match (NEAR_RANK == 1) and removes modeling type 0 (walkways) from the analysis. This is because the walkways are often the least reliably georectified segments of any network. 

In [20]:
modeling_type_to_osm_type = modeling_type_to_osm_type[modeling_type_to_osm_type['NEAR_RANK'] == 1]
modeling_type_to_osm_type = modeling_type_to_osm_type[modeling_type_to_osm_type['modeling_type'] != 0]
modeling_type_to_osm_type.head()

,osm_id,osm_type,modeling_id,modeling_type,FID,distance,NEAR_RANK,NEAR_ANGLE,FROM_X,FROM_Y,NEAR_X,NEAR_Y
8,0,motorway,1333,1,19154,2.761085,1,-76.547166,-105.015621,39.733871,-105.015652,39.733877
9,0,motorway,1333,1,19157,1.700434,1,-81.442178,-105.015775,39.733308,-105.015795,39.733310
10,0,motorway,1333,1,19160,2.763084,1,-87.108449,-105.015831,39.732725,-105.015864,39.732727
11,0,motorway,1333,1,19163,2.603804,1,-91.150870,-105.015851,39.732141,-105.015882,39.732141
13,1,motorway,2187,1,26014,2.654804,1,-124.441124,-104.932644,39.676962,-104.932670,39.676949


The cell below is used to count the instances of each type of match between the networks, as mentioned earlier the one of primary importance is type of match that is most commonly matched to modeling network segments with modeling type of 1.

In [22]:
modeling_type_to_osm_type = modeling_type_to_osm_type.filter(['osm_type', 'modeling_type'])

count_of_types = modeling_type_to_osm_type.groupby(['osm_type','modeling_type']).size().reset_index().rename(columns={0:'count'})
count_of_types = count_of_types.sort_values(by='count', ascending=False)
count_by_modeling_type = count_of_types.sort_values(by = 'modeling_type')

count_by_modeling_type

,osm_type,modeling_type,count
22,residential,1,1
38,tertiary,1,40
18,primary_link,1,1
28,secondary,1,10
7,motorway_link,1,140
13,primary,1,3
1,motorway,1,2147
2,motorway,2,15
39,tertiary,2,14
29,secondary,2,13
